#### concepts to know about
- partial
- __call__ for classes
- *args and **kwargs
- callbacks
- __dunder__ thingies

In [1]:
import torch
import matplotlib.pyplot as plt
import random

### Callbacks

In [2]:
import ipywidgets as widgets

In [3]:
w = widgets.Button(description='Click me')

In [4]:
def f(o): print('hi')

In [5]:
w.on_click(f)

In [6]:
w

Button(description='Click me', style=ButtonStyle())

In [7]:
w.click()

hi


In [8]:
from time import sleep

In [9]:
def slow_calculation():
    res = 0
    for i in range(5):
        res += i*i
        sleep(1)
    return res

In [10]:
slow_calculation()

30

In [11]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        res += i*i
        sleep(1)
        if cb:
            cb(i)
    return res

In [12]:
def show_progress(epoch):
    print(f'Awesome! Epoch: {epoch}')

In [13]:
slow_calculation(show_progress)

Awesome! Epoch: 0
Awesome! Epoch: 1
Awesome! Epoch: 2
Awesome! Epoch: 3
Awesome! Epoch: 4


30

In [14]:
slow_calculation(lambda epoch: print(f'Awesome! Epoch: {epoch}'))

Awesome! Epoch: 0
Awesome! Epoch: 1
Awesome! Epoch: 2
Awesome! Epoch: 3
Awesome! Epoch: 4


30

In [15]:
def show_progress(exclamation, epoch):
    print(f'Awesome! {exclamation}, epoch: {epoch}')

In [16]:
slow_calculation(lambda epoch: show_progress('Job', epoch))

Awesome! Job, epoch: 0
Awesome! Job, epoch: 1
Awesome! Job, epoch: 2
Awesome! Job, epoch: 3
Awesome! Job, epoch: 4


30

In [17]:
def f(a, b, c):
    print(a, b, c)

In [18]:
from functools import partial

In [19]:
f(1, 2, 3)

1 2 3


In [20]:
g = partial(f, 1, 2)

In [21]:
g(3)

1 2 3


In [22]:
def make_show_progress(exclamation):
    def _inner(epoch):
        print(f'Awesome job {exclamation}, epoch: {epoch}')
    return _inner

In [23]:
slow_calculation(make_show_progress("Nice"))

Awesome job Nice, epoch: 0
Awesome job Nice, epoch: 1
Awesome job Nice, epoch: 2
Awesome job Nice, epoch: 3
Awesome job Nice, epoch: 4


30

In [24]:
slow_calculation(partial(show_progress, 'Nice'))

Awesome! Nice, epoch: 0
Awesome! Nice, epoch: 1
Awesome! Nice, epoch: 2
Awesome! Nice, epoch: 3
Awesome! Nice, epoch: 4


30

### Callbacks as callable classes

In [25]:
class ProgressShowingCallback():
    def __init__(self, exclamation): self.exclamation = exclamation
    def __call__(self, epoch):
        print(f'Awesome! {self.exclamation}, epoch: {epoch}')

In [26]:
cb = ProgressShowingCallback("Nice")

In [27]:
cb(1)

Awesome! Nice, epoch: 1


In [28]:
slow_calculation(cb)

Awesome! Nice, epoch: 0
Awesome! Nice, epoch: 1
Awesome! Nice, epoch: 2
Awesome! Nice, epoch: 3
Awesome! Nice, epoch: 4


30

### Multiple callback funcs; *args and **kwargs

In [29]:
def f(*args, **kwargs):
    # print(type(args), type(kwargs))
    print(f"args: {args}, kwargs: {kwargs}")

In [30]:
f(3, 'a', thing1='hello')

args: (3, 'a'), kwargs: {'thing1': 'hello'}


In [31]:
def g(a, b, c=0):
    print(a, b, c)

In [32]:
args=[1,2]
kwargs={'c': 3}
g(*args, **kwargs)

1 2 3


- Passing args and kwargs
- Functions which accept these as params
- Can be used to abosrb all the unused args as well

In [33]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        if cb: cb.before_calc(i)
        res += i*i
        sleep(1)
        if cb: cb.after_calc(i, res)
    
    return res

In [34]:
class PrintStepCallback():
    def before_calc(self, *args, **kwargs): print('About to start')
    def after_calc(self, *args, **kwarfs): print('Done')

In [35]:
slow_calculation(PrintStepCallback())

About to start
Done
About to start
Done
About to start
Done
About to start
Done
About to start
Done


30

In [36]:
class PrintStatusCallback():
    def before_calc(self, *args, **kwargs): print('About to start')
    def after_calc(self, *args, **kwargs): print(f"Epoch: {args[0]}, val: {args[1]}")

In [37]:
slow_calculation(PrintStatusCallback())

About to start
Epoch: 0, val: 0
About to start
Epoch: 1, val: 1
About to start
Epoch: 2, val: 5
About to start
Epoch: 3, val: 14
About to start
Epoch: 4, val: 30


30

In [38]:
class SlowCalculator():
    def __init__(self, cb=None):
        self.cb,self.res = cb, 0
        
    def callback(self, cb_name, *args):
        if not self.cb:
            return
        cb = getattr(self.cb, cb_name, None)
        if cb: return cb(self, *args)
    
    def calc(self):
        for i in range(5):
            self.callback('before_calc', i)
            self.res += i * i
            sleep(1)
            if self.callback('after_calc', i):
                print('early stop')
                break

In [39]:
class ModifyingCallback():
    def after_calc(self, calc, epoch):
        print(f'After {epoch}: {calc.res}')
        if calc.res > 10:
            return True
        if calc.res < 3:
            calc.res = calc.res * 2
        

In [40]:
c = SlowCalculator(ModifyingCallback())

In [41]:
c.calc()

After 0: 0
After 1: 1
After 2: 6
After 3: 15
early stop


In [42]:
c.res

15

In [43]:
getattr(c, 'calc')()

After 0: 15
early stop


In [44]:
class SloppyAdder():
    def __init__(self, o): self.o=o
    def __add__(self, s): return SloppyAdder(self.o + s.o + 0.01)
    def __repr__(self): return str(self.o)

In [45]:
s1 = SloppyAdder(1)
s2 = SloppyAdder(2)
s1+s2

3.01

In [46]:
class B:
    a,b=1,2
    def __getattr__(self, k):
        if k[0]=='_': raise AttributeError(k)
        return f'Hello {k}'
    
    def abcd(self):
        return 'abcd'

In [47]:
b = B()

In [48]:
b.a

1

In [49]:
b.foo

'Hello foo'

In [50]:
getattr(b, 'a')

1

In [51]:
getattr(b, 'abc')

'Hello abc'

In [52]:
b.abcd()

'abcd'

In [53]:
hasattr(b, 'abcd')

True

In [54]:
getattr(b, 'a')

1

In [55]:
getattr(b, 'b')

2

In [56]:
getattr(b, 'abc')

'Hello abc'